# CGAN_MNIST_with_Keras

https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py

# 1. Load Modules

In [24]:
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt

# 2. Build Network

In [35]:
class CGAN():
    
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.img_channels = 1
        self.img_shape = (self.img_rows,self.img_cols,self.img_channels)
        self.num_classes = 10
        self.latent_dim = 100
        
        optimizer = Adam(0.0002,0.5)
        
        # Build and compile D
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                  optimizer=optimizer,
                                  metrics=['accuracy'])
        
        # Build G
        self.generator = self.build_generator()
        
        #  G <- noise & target label as input
        # G corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        # 다른 GAN과 다른 부분이다. label이라는 '논문에서 y'에 해당하는 condition을 추가해준다. 
        img = self.generator([noise,label])
        
        # Combined model에 대해서는 G만 학습할 것이다. 
        # 그래서 D는 training = False로 설정
        self.discriminator.trainable = False
        
        # D <- generated imgs as inputs and determines Validity
        # label of that imgs
        valid = self.discriminator([img,label])
        
        
        # Combined model
        # Trains G to fool D
        self.combined = Model([noise,label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
                             optimizer=optimizer)  ### ? 왜 metrics가 안들어가는지
        
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(256,input_dim = self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape),activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,),dtype='int32')
        ## 이 부분 코딩 ()() 형태 무엇인지
        # label을 vector에 embedding 함
        label_embedding = Flatten()(Embedding(self.num_classes,self.latent_dim)(label))
        
        model_input = multiply([noise,label_embedding])
        img = model(model_input)
        
        return Model([noise,label],img)
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Dense(512,input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1,activation='sigmoid'))
        model.summary()
        
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,),dtype='int32')
        
        label_embedding = Flatten()(Embedding(self.num_classes,np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)
        
        model_input = multiply([flat_img,label_embedding])
        
        validity = model(model_input)
        
        return Model([img,label],validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        # Load MNIST
        (X_train,y_train),(_,_) = mnist.load_data()
        
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train,axis=3)
        y_train = y_train.reshape(-1,1)
        
        # Adversarial Ground Truth
        valid = np.ones((batch_size,1))
        fake = np.zeros((batch_size,1))
        
        for epoch in range(epochs):
            """Train D"""
            idx = np.random.randint(0,X_train.shape[0],batch_size)
            imgs,labels = X_train[idx], y_train[idx]
            
            # Sample noise as generator input
            noise = np.random.normal(0,1,(batch_size,100))
            
            # Generate a half batch of new imgs
            gen_imgs = self.generator.predict([noise,labels])
            
            # Train D
            d_loss_real = self.discriminator.train_on_batch([imgs,labels],valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs,labels],fake)
            d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
            
            
            """Train G"""
            
            sampled_labels = np.random.randint(0,10,batch_size).reshape(-1,1)
            
            # Train G
            g_loss = self.combined.train_on_batch([noise,sampled_labels],valid)
            
            # Plot the progress
            print("Epoch : {0} /D_Loss : {1} / ACC : {2:.2f} / G_Loss : {3}".format(epoch,d_loss[0],d_loss[1]*100,g_loss))
            
            # Interval
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                
                
    def sample_images(self,epoch):
        r,c = 2, 5
        noise = np.random.normal(0,1,(r*c,100))
        sampled_labels = np.arange(0,10).reshape(-1,1)
        
        gen_imgs = self.generator.predict([noise,sampled_labels])
        
        # Rescale
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig,axs = plt.subplots(r,c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
                axs[i,j].set_title("Digit:%d"% sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
        
        
            
        
        

# 3. Train Model 

In [37]:
if __name__ == "__main__":
    cgan = CGAN()
    cgan.train(epochs=20000,batch_size=32,sample_interval=400)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_55 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_74 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_56 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_57 (LeakyReLU)   (None, 512)               0         
__________

Epoch : 61 /D_Loss : 0.07201477885246277 / ACC : 96.88 / G_Loss : 6.270431041717529
Epoch : 62 /D_Loss : 0.035786762833595276 / ACC : 100.00 / G_Loss : 6.383002281188965
Epoch : 63 /D_Loss : 0.03404606878757477 / ACC : 100.00 / G_Loss : 6.038084983825684
Epoch : 64 /D_Loss : 0.028150200843811035 / ACC : 100.00 / G_Loss : 6.486711502075195
Epoch : 65 /D_Loss : 0.12943395972251892 / ACC : 92.19 / G_Loss : 6.666329860687256
Epoch : 66 /D_Loss : 0.21347260475158691 / ACC : 95.31 / G_Loss : 7.27717399597168
Epoch : 67 /D_Loss : 0.08614687621593475 / ACC : 95.31 / G_Loss : 7.15479850769043
Epoch : 68 /D_Loss : 0.3703703284263611 / ACC : 85.94 / G_Loss : 5.420538902282715
Epoch : 69 /D_Loss : 0.2518104612827301 / ACC : 87.50 / G_Loss : 6.321256637573242
Epoch : 70 /D_Loss : 0.04440942779183388 / ACC : 98.44 / G_Loss : 7.2203216552734375
Epoch : 71 /D_Loss : 0.10483258962631226 / ACC : 95.31 / G_Loss : 4.977363586425781
Epoch : 72 /D_Loss : 0.12637458741664886 / ACC : 95.31 / G_Loss : 6.152075

Epoch : 159 /D_Loss : 0.38380590081214905 / ACC : 82.81 / G_Loss : 5.717267990112305
Epoch : 160 /D_Loss : 0.3714304268360138 / ACC : 78.12 / G_Loss : 4.316250324249268
Epoch : 161 /D_Loss : 0.19619107246398926 / ACC : 92.19 / G_Loss : 4.017922878265381
Epoch : 162 /D_Loss : 0.228617325425148 / ACC : 93.75 / G_Loss : 4.667015552520752
Epoch : 163 /D_Loss : 0.24644920229911804 / ACC : 89.06 / G_Loss : 5.836092472076416
Epoch : 164 /D_Loss : 0.3278067708015442 / ACC : 82.81 / G_Loss : 4.73037052154541
Epoch : 165 /D_Loss : 0.19439081847667694 / ACC : 95.31 / G_Loss : 5.538298606872559
Epoch : 166 /D_Loss : 0.5195645689964294 / ACC : 70.31 / G_Loss : 5.296663284301758
Epoch : 167 /D_Loss : 0.2579866647720337 / ACC : 92.19 / G_Loss : 6.2453227043151855
Epoch : 168 /D_Loss : 0.29852965474128723 / ACC : 89.06 / G_Loss : 5.766251564025879
Epoch : 169 /D_Loss : 0.41036027669906616 / ACC : 85.94 / G_Loss : 5.133580207824707
Epoch : 170 /D_Loss : 0.4370133876800537 / ACC : 79.69 / G_Loss : 3.980

Epoch : 259 /D_Loss : 0.49310049414634705 / ACC : 75.00 / G_Loss : 4.6746826171875
Epoch : 260 /D_Loss : 0.339890718460083 / ACC : 85.94 / G_Loss : 2.9337830543518066
Epoch : 261 /D_Loss : 0.41041141748428345 / ACC : 81.25 / G_Loss : 4.048304557800293
Epoch : 262 /D_Loss : 0.3865286707878113 / ACC : 79.69 / G_Loss : 3.099410057067871
Epoch : 263 /D_Loss : 0.5010160803794861 / ACC : 75.00 / G_Loss : 3.8736987113952637
Epoch : 264 /D_Loss : 0.3917267918586731 / ACC : 79.69 / G_Loss : 3.5726213455200195
Epoch : 265 /D_Loss : 0.45247894525527954 / ACC : 75.00 / G_Loss : 3.4926741123199463
Epoch : 266 /D_Loss : 0.5208369493484497 / ACC : 68.75 / G_Loss : 3.0272469520568848
Epoch : 267 /D_Loss : 0.47357016801834106 / ACC : 82.81 / G_Loss : 2.891077995300293
Epoch : 268 /D_Loss : 0.5473586320877075 / ACC : 73.44 / G_Loss : 3.099885940551758
Epoch : 269 /D_Loss : 0.3816755414009094 / ACC : 85.94 / G_Loss : 3.584714412689209
Epoch : 270 /D_Loss : 0.5390151739120483 / ACC : 73.44 / G_Loss : 3.67

Epoch : 357 /D_Loss : 0.4553050994873047 / ACC : 78.12 / G_Loss : 2.6165804862976074
Epoch : 358 /D_Loss : 0.4422653019428253 / ACC : 82.81 / G_Loss : 2.2860891819000244
Epoch : 359 /D_Loss : 0.4313053488731384 / ACC : 81.25 / G_Loss : 1.9730814695358276
Epoch : 360 /D_Loss : 0.4550495743751526 / ACC : 85.94 / G_Loss : 2.4519448280334473
Epoch : 361 /D_Loss : 0.47419190406799316 / ACC : 78.12 / G_Loss : 2.6245365142822266
Epoch : 362 /D_Loss : 0.5544700622558594 / ACC : 70.31 / G_Loss : 2.2592673301696777
Epoch : 363 /D_Loss : 0.37098902463912964 / ACC : 82.81 / G_Loss : 2.32365083694458
Epoch : 364 /D_Loss : 0.6427009105682373 / ACC : 65.62 / G_Loss : 2.1523585319519043
Epoch : 365 /D_Loss : 0.6399616003036499 / ACC : 65.62 / G_Loss : 2.168487071990967
Epoch : 366 /D_Loss : 0.5843888521194458 / ACC : 75.00 / G_Loss : 3.071589469909668
Epoch : 367 /D_Loss : 0.5716670751571655 / ACC : 62.50 / G_Loss : 2.248878240585327
Epoch : 368 /D_Loss : 0.4763696491718292 / ACC : 84.38 / G_Loss : 2.

...

Epoch : 19574 /D_Loss : 0.6595368385314941 / ACC : 60.94 / G_Loss : 0.8545457124710083
Epoch : 19575 /D_Loss : 0.6451194882392883 / ACC : 57.81 / G_Loss : 0.8053115606307983
Epoch : 19576 /D_Loss : 0.6830331683158875 / ACC : 62.50 / G_Loss : 0.8352411985397339
Epoch : 19577 /D_Loss : 0.6672596335411072 / ACC : 60.94 / G_Loss : 0.8039398193359375
Epoch : 19578 /D_Loss : 0.6898376941680908 / ACC : 53.12 / G_Loss : 0.8196926712989807
Epoch : 19579 /D_Loss : 0.6197518110275269 / ACC : 65.62 / G_Loss : 0.83139568567276
Epoch : 19580 /D_Loss : 0.6357283592224121 / ACC : 57.81 / G_Loss : 0.7311631441116333
Epoch : 19581 /D_Loss : 0.6476038098335266 / ACC : 51.56 / G_Loss : 0.7688794136047363
Epoch : 19582 /D_Loss : 0.6267836093902588 / ACC : 67.19 / G_Loss : 0.832485556602478
Epoch : 19583 /D_Loss : 0.7085772752761841 / ACC : 50.00 / G_Loss : 0.8184897899627686
Epoch : 19584 /D_Loss : 0.6731737852096558 / ACC : 59.38 / G_Loss : 0.8079034686088562
Epoch : 19585 /D_Loss : 0.6387879848480225 / A

Epoch : 19671 /D_Loss : 0.6418824195861816 / ACC : 60.94 / G_Loss : 0.833446204662323
Epoch : 19672 /D_Loss : 0.7053476572036743 / ACC : 51.56 / G_Loss : 0.8561343550682068
Epoch : 19673 /D_Loss : 0.6795682907104492 / ACC : 54.69 / G_Loss : 0.8139971494674683
Epoch : 19674 /D_Loss : 0.6938210129737854 / ACC : 54.69 / G_Loss : 0.8196243643760681
Epoch : 19675 /D_Loss : 0.6622772216796875 / ACC : 51.56 / G_Loss : 0.8486028909683228
Epoch : 19676 /D_Loss : 0.6191329956054688 / ACC : 73.44 / G_Loss : 0.8226298689842224
Epoch : 19677 /D_Loss : 0.6821834444999695 / ACC : 46.88 / G_Loss : 0.7868491411209106
Epoch : 19678 /D_Loss : 0.6766706705093384 / ACC : 54.69 / G_Loss : 0.8192600011825562
Epoch : 19679 /D_Loss : 0.6594476699829102 / ACC : 56.25 / G_Loss : 0.8310623168945312
Epoch : 19680 /D_Loss : 0.6499392986297607 / ACC : 57.81 / G_Loss : 0.8730218410491943
Epoch : 19681 /D_Loss : 0.6679173707962036 / ACC : 54.69 / G_Loss : 0.8463135361671448
Epoch : 19682 /D_Loss : 0.6919811964035034 /

Epoch : 19767 /D_Loss : 0.638748824596405 / ACC : 65.62 / G_Loss : 0.8615202903747559
Epoch : 19768 /D_Loss : 0.6357712149620056 / ACC : 64.06 / G_Loss : 0.7871447205543518
Epoch : 19769 /D_Loss : 0.711532473564148 / ACC : 54.69 / G_Loss : 0.8255009651184082
Epoch : 19770 /D_Loss : 0.6540144681930542 / ACC : 67.19 / G_Loss : 0.8182107210159302
Epoch : 19771 /D_Loss : 0.7047320604324341 / ACC : 54.69 / G_Loss : 0.9039878845214844
Epoch : 19772 /D_Loss : 0.6463778018951416 / ACC : 62.50 / G_Loss : 0.8277154564857483
Epoch : 19773 /D_Loss : 0.6695460081100464 / ACC : 54.69 / G_Loss : 0.836284339427948
Epoch : 19774 /D_Loss : 0.6748605370521545 / ACC : 65.62 / G_Loss : 0.8308111429214478
Epoch : 19775 /D_Loss : 0.6296724081039429 / ACC : 65.62 / G_Loss : 0.8220443725585938
Epoch : 19776 /D_Loss : 0.6488728523254395 / ACC : 65.62 / G_Loss : 0.8156189918518066
Epoch : 19777 /D_Loss : 0.6853355169296265 / ACC : 60.94 / G_Loss : 0.8723429441452026
Epoch : 19778 /D_Loss : 0.6372923851013184 / A

Epoch : 19865 /D_Loss : 0.6884347200393677 / ACC : 50.00 / G_Loss : 0.8178650140762329
Epoch : 19866 /D_Loss : 0.6580544710159302 / ACC : 59.38 / G_Loss : 0.7361796498298645
Epoch : 19867 /D_Loss : 0.6335403919219971 / ACC : 54.69 / G_Loss : 0.8558201789855957
Epoch : 19868 /D_Loss : 0.6627210378646851 / ACC : 67.19 / G_Loss : 0.7461227178573608
Epoch : 19869 /D_Loss : 0.6988145709037781 / ACC : 50.00 / G_Loss : 0.7893463373184204
Epoch : 19870 /D_Loss : 0.669748067855835 / ACC : 53.12 / G_Loss : 0.7848294973373413
Epoch : 19871 /D_Loss : 0.6711223721504211 / ACC : 59.38 / G_Loss : 0.8002027273178101
Epoch : 19872 /D_Loss : 0.6314405798912048 / ACC : 57.81 / G_Loss : 0.812117338180542
Epoch : 19873 /D_Loss : 0.7015484571456909 / ACC : 50.00 / G_Loss : 0.8236331939697266
Epoch : 19874 /D_Loss : 0.7085652351379395 / ACC : 50.00 / G_Loss : 0.8271979689598083
Epoch : 19875 /D_Loss : 0.6180864572525024 / ACC : 64.06 / G_Loss : 0.8024107217788696
Epoch : 19876 /D_Loss : 0.6781620979309082 / 

Epoch : 19963 /D_Loss : 0.6680210828781128 / ACC : 51.56 / G_Loss : 0.8974266052246094
Epoch : 19964 /D_Loss : 0.6536256670951843 / ACC : 68.75 / G_Loss : 0.8010903596878052
Epoch : 19965 /D_Loss : 0.6619270443916321 / ACC : 56.25 / G_Loss : 0.8381459712982178
Epoch : 19966 /D_Loss : 0.6542036533355713 / ACC : 67.19 / G_Loss : 0.8381608724594116
Epoch : 19967 /D_Loss : 0.6528668403625488 / ACC : 62.50 / G_Loss : 0.864685595035553
Epoch : 19968 /D_Loss : 0.6721776723861694 / ACC : 56.25 / G_Loss : 0.863638162612915
Epoch : 19969 /D_Loss : 0.6116108894348145 / ACC : 60.94 / G_Loss : 0.8076707124710083
Epoch : 19970 /D_Loss : 0.627852737903595 / ACC : 64.06 / G_Loss : 0.8949062824249268
Epoch : 19971 /D_Loss : 0.6484256386756897 / ACC : 59.38 / G_Loss : 0.8418935537338257
Epoch : 19972 /D_Loss : 0.6593418121337891 / ACC : 50.00 / G_Loss : 0.8687275648117065
Epoch : 19973 /D_Loss : 0.6850378513336182 / ACC : 45.31 / G_Loss : 0.7977966070175171
Epoch : 19974 /D_Loss : 0.6611950993537903 / A

# 4. Make GIF

In [48]:
for i in filename:
    a = i.split('/')[-1]
    leng = re.findall('\d+',a)[0]
    leng = "0"*(5-len(leng))+leng
    print(leng)

06400
13600
08000
11200
04000
16800
00400
14800
18800
08400
11600
04400
06000
13200
10800
02000
17200
15600
19600
15200
19200
02400
17600
12800
12400
07600
05200
10000
09200
02800
05600
10400
09600
12000
07200
04800
08800
16000
03200
18400
00800
01600
14400
18000
01200
14000
16400
03600
00000
06800


In [54]:
# 파일이름의 숫자 5자리로 만들어서 정렬하기
import re, glob
path = os.getcwd()
path2 = path + '/images'
filename = glob.glob(path + '/images/*.png')
for i in filename:
    a = i.split('/')[-1]
    leng = re.findall('\d+',a)[0]
    leng = "0"*(5-len(leng))+leng
    os.rename(i,leng+".png")

In [55]:
filename_new = glob.glob(path + '/images/*.png')
filename_new = sorted(filename_new)
filename_new[:5]
# 순서대로 정렬된 것을 볼 수 있다. 

['/Users/charming/Python/0_Paper_Review/10. CGAN/images/00000.png',
 '/Users/charming/Python/0_Paper_Review/10. CGAN/images/00400.png',
 '/Users/charming/Python/0_Paper_Review/10. CGAN/images/00800.png',
 '/Users/charming/Python/0_Paper_Review/10. CGAN/images/01200.png',
 '/Users/charming/Python/0_Paper_Review/10. CGAN/images/01600.png']

In [56]:
impor
generated_image_array = [imageio.imread(generated_image) for generated_image in filename_new]
imageio.mimsave('CGAN_MNIST.gif', generated_image_array, fps=10)

NameError: name 'imageio' is not defined